In [1]:
import numpy as np
from pyquil.quil import Program, Pragma
from pyquil.api import get_qc
from pyquil.gates import *
import numpy as np

In [6]:
p = Program()
ro = p.declare('ro', 'BIT', 1)
p += H(0)
p += Pragma('DELAY', [0], str(200e-9))
p += MEASURE(0, ro[0])
print(p)
qc = get_qc('4q-qvm', execution_timeout=1000)
p = qc.compiler.quil_to_native_quil(p)
print(p)
qc.qam.gate_noise=(0.2, 0.0, 0.0)
qc.run(p)


DECLARE ro BIT[1]
H 0
PRAGMA DELAY 0 "2e-07"
MEASURE 0 ro[0]

DECLARE ro BIT[1]
RZ(pi) 0
RX(pi/2) 0
RZ(pi/2) 0
RX(-pi/2) 0
MEASURE 0 ro[0]
HALT



QAMExecutionResult(executable=<pyquil.quil.Program object at 0x000001CAF379F8D0>, readout_data={'ro': array([[0]])}, execution_duration_microseconds=None)

In [ ]:
# new thin implementation try
def add_noise(
		qc: QuantumComputer,
		p: Program,
		convert_to_native: bool = True,
		calibrations: Optional[Calibrations] = None,
		depolarizing: bool = False,
		damping_and_dephasing: Optional[int] = 2,
		# off= None, 0 = only on targets, 1 = on all qubits after every 1q, 2 = on all qubits after every 2q
		readout: bool = True,
		noise_intensity: float = 1,
) -> Program:
	if convert_to_native:
		p = qc.compiler.quil_to_native_quil(p)
	qubits = p.get_qubits()

	defenitions = []
	new_p = Program()
	for i in p:
		if (damping_and_dephasing is not None) and ((isinstance(i, Pragma) and i.command == "DELAY") or (isinstance(i, Gate)) and i.name == "DELAY"):

		if isinstance(i, Gate):
			targets = tuple(t.index for t in i.qubits)
			# for 2-qubit gates, add decoherence noise for all qubits in qc
			if len(targets) == 2:
				if noise_types.fidelity:
					new_p += noise_types.gates["Depolarising_" + i.name](targets)
				if noise_types.decoherence_2Q:
					for q in qubits:
						if q not in targets or not noise_types.fidelity:
							new_p += noise_types.gates[Noisy_I_2Q_name](q)
			elif len(targets) == 1:
				if noise_types.fidelity:
					for q in targets:
						# new_p += Noisy_I_1Q_fidelity(q)
						continue
				if noise_types.decoherence_1Q:
					for q in qubits:
						if q not in targets or not noise_types.fidelity:
							new_p += noise_types.gates[Noisy_I_1Q_name](q)

	# deal with delay:
	new_p, delay_gates = replace_delay_with_noisy_I(new_p)

	if calibrations is None:
		calibrations = Calibrations(qc=qc)

	# add kraus maps declarations:
	if noise_types.decoherence_1Q or noise_types.decoherence_2Q:
		new_p = add_decoherence_noise(prog=new_p, T1=calibrations.T1, T2=calibrations.T2)
	if noise_types.readout:
		new_p = add_readout_noise(prog=new_p, ro_fidelity=calibrations.readout)

	for gate in delay_gates:
		add_delay_maps(new_p, gate, calibrations.T1, calibrations.T2)

	new_p.wrap_in_numshots_loop(p.num_shots)  # wrap in original program's numshots
	return new_p
